In [1]:
import warnings
import pprint
import skrebate
import imblearn
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import r2_score, auc, roc_auc_score, balanced_accuracy_score, confusion_matrix
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from xgboost import XGBRegressor, XGBClassifier

warnings.simplefilter('ignore')



In [2]:
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [3]:
work_dir = './mIHC/'

In [4]:
test3_master = pd.read_csv(work_dir + 'test3/test3_masterset.csv', sep=',', index_col=0)
test3_master

,Intensity_IntegratedIntensityEdge_CD138_rs,Intensity_IntegratedIntensityEdge_CD20_rs,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,Intensity_IntegratedIntensityEdge_CD45_rs,Intensity_IntegratedIntensityEdge_CD4_rs,Intensity_IntegratedIntensityEdge_CD5_rs,Intensity_IntegratedIntensityEdge_CD68_rs,Intensity_IntegratedIntensityEdge_CD8_rs,...,Intensity_UpperQuartileIntensity_PANCK_rs,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs,Location_Center_X,Location_Center_Y,obj_ID,AreaShape_Area,class
1,0.008000,0.000000,0.025000,0.000000,0.018182,0.018349,0.272727,0.015385,0.000000,0.012270,...,0.007353,0.017857,0.000000,0.000000,0.000000,24.500000,1.285714,SMA06F_1,14,G
2,0.120000,0.015385,1.808333,0.255682,4.818182,6.128440,4.383838,3.374359,0.004695,0.184049,...,0.000000,0.044643,0.036585,0.034188,0.004608,36.521739,3.144928,SMA06F_2,69,B
3,0.376000,0.569231,9.591667,0.187500,6.642424,10.055046,5.979798,2.651282,0.046948,0.190184,...,0.004902,0.473214,0.012195,0.094017,0.004608,110.648148,5.055556,SMA06F_3,108,B
4,0.064000,0.023077,0.258333,0.096591,4.369697,3.587156,1.323232,1.266667,0.009390,0.024540,...,0.000000,0.008929,0.000000,0.034188,0.000000,130.466667,3.746667,SMA06F_4,75,B
5,0.168000,0.030769,0.433333,0.011364,6.466667,2.229358,3.181818,8.225641,0.436620,0.006135,...,0.004902,0.102679,0.018293,0.038462,0.200461,149.354545,6.881818,SMA06F_5,110,B
6,0.584000,0.315385,0.458333,0.017045,0.284848,0.275229,1.191919,0.333333,0.000000,0.184049,...,0.186275,0.017857,0.048780,0.000000,0.004608,310.659574,2.851064,SMA06F_6,94,H
7,1.864000,0.015385,0.325000,0.017045,0.181818,0.137615,1.434343,0.076923,0.009390,0.061350,...,0.246324,0.035714,0.024390,0.000000,0.000000,343.705263,6.473684,SMA06F_7,95,H
8,0.128000,0.015385,0.183333,0.068182,1.557576,2.073395,2.727273,0.230769,2.788732,0.036810,...,0.009804,0.035714,0.036585,0.000000,0.000000,378.138889,5.097222,SMA06F_8,72,D
9,0.552000,0.000000,0.058333,0.000000,2.878788,2.009174,0.151515,0.271795,0.018779,3.644172,...,0.066176,0.053571,0.000000,0.051282,0.000000,423.500000,1.363636,SMA06F_9,22,G
10,1.752000,0.023077,0.066667,0.005682,0.187879,0.064220,0.626263,0.148718,0.028169,0.128834,...,0.193627,0.062500,0.033537,0.000000,0.000000,658.682927,2.341463,SMA06F_10,41,H


In [5]:
# remove duplicated
test3_master = test3_master.loc[~test3_master.duplicated()]
# generate patient ID
test3_master['patient_ID'] = test3_master['obj_ID'].apply(lambda x: x.split('_')[0]).copy()
# re-order columns
columns = test3_master.columns.tolist()
columns[0], columns[362] = columns[362], columns[0]
columns[1], columns[365] = columns[365], columns[1]
columns[2], columns[364] = columns[364], columns[2]
test3_master_new = test3_master[columns]
test3_master_new = test3_master_new.fillna(0.)
test3_master_new

,obj_ID,patient_ID,class,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,Intensity_IntegratedIntensityEdge_CD45_rs,Intensity_IntegratedIntensityEdge_CD4_rs,Intensity_IntegratedIntensityEdge_CD5_rs,Intensity_IntegratedIntensityEdge_CD68_rs,Intensity_IntegratedIntensityEdge_CD8_rs,...,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs,Location_Center_X,Location_Center_Y,Intensity_IntegratedIntensityEdge_CD138_rs,AreaShape_Area,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD20_rs
1,SMA06F_1,SMA06F,G,0.000000,0.018182,0.018349,0.272727,0.015385,0.000000,0.012270,...,0.017857,0.000000,0.000000,0.000000,24.500000,1.285714,0.008000,14,0.025000,0.000000
2,SMA06F_2,SMA06F,B,0.255682,4.818182,6.128440,4.383838,3.374359,0.004695,0.184049,...,0.044643,0.036585,0.034188,0.004608,36.521739,3.144928,0.120000,69,1.808333,0.015385
3,SMA06F_3,SMA06F,B,0.187500,6.642424,10.055046,5.979798,2.651282,0.046948,0.190184,...,0.473214,0.012195,0.094017,0.004608,110.648148,5.055556,0.376000,108,9.591667,0.569231
4,SMA06F_4,SMA06F,B,0.096591,4.369697,3.587156,1.323232,1.266667,0.009390,0.024540,...,0.008929,0.000000,0.034188,0.000000,130.466667,3.746667,0.064000,75,0.258333,0.023077
5,SMA06F_5,SMA06F,B,0.011364,6.466667,2.229358,3.181818,8.225641,0.436620,0.006135,...,0.102679,0.018293,0.038462,0.200461,149.354545,6.881818,0.168000,110,0.433333,0.030769
6,SMA06F_6,SMA06F,H,0.017045,0.284848,0.275229,1.191919,0.333333,0.000000,0.184049,...,0.017857,0.048780,0.000000,0.004608,310.659574,2.851064,0.584000,94,0.458333,0.315385
7,SMA06F_7,SMA06F,H,0.017045,0.181818,0.137615,1.434343,0.076923,0.009390,0.061350,...,0.035714,0.024390,0.000000,0.000000,343.705263,6.473684,1.864000,95,0.325000,0.015385
8,SMA06F_8,SMA06F,D,0.068182,1.557576,2.073395,2.727273,0.230769,2.788732,0.036810,...,0.035714,0.036585,0.000000,0.000000,378.138889,5.097222,0.128000,72,0.183333,0.015385
9,SMA06F_9,SMA06F,G,0.000000,2.878788,2.009174,0.151515,0.271795,0.018779,3.644172,...,0.053571,0.000000,0.051282,0.000000,423.500000,1.363636,0.552000,22,0.058333,0.000000
10,SMA06F_10,SMA06F,H,0.005682,0.187879,0.064220,0.626263,0.148718,0.028169,0.128834,...,0.062500,0.033537,0.000000,0.000000,658.682927,2.341463,1.752000,41,0.066667,0.023077


In [6]:
# data imbalance analysis
test3_master_new['patient_ID'].value_counts(), test3_master_new['class'].value_counts()

(SMA19F    53773
 SMA37F    45614
 SMA40F    21363
 SMA06F    17459
 SMA54F    13718
 SMA60F    11566
 SMA21F     9476
 Name: patient_ID, dtype: int64, F    113129
 H     26684
 G     14324
 E      9796
 B      3899
 A      3025
 D      1391
 C       721
 Name: class, dtype: int64)

In [7]:
# function
def sample_1rest(patient_id, sample_class, sample_sizes):
    print(patient_id)
    patient = test3_master_new[test3_master_new['patient_ID'] == patient_id]
    print(patient['class'].value_counts().to_frame())
    
    classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    if len(sample_sizes) != len(classes):
        raise Exception("Error: sample_sizes don't match classes.")
    
    rval = None
    for idx, class_id in enumerate(classes):
        patient_one_class = patient[patient['class'] == class_id]
        

        patient_one_class_sample =patient_one_class.iloc[np.random.choice(patient_one_class.shape[0], sample_sizes[idx], replace=False)]
        if rval is None:
            rval = patient_one_class_sample
        else:
            rval = rval.append(patient_one_class_sample, ignore_index=True)
    print(rval['patient_ID'].value_counts().to_frame())
    print(rval['class'].value_counts().to_frame())
    rval.insert(loc=3, column='b_class', value='others')
    rval.loc[rval['class']==sample_class, 'b_class'] = sample_class
    print(rval['patient_ID'].value_counts().to_frame())
    print(rval['b_class'].value_counts().to_frame())
    return rval

## per patient

In [8]:
# patient 'SMA19F'
patient_SMA19F = test3_master_new[test3_master_new['patient_ID'] == 'SMA19F']
patient_SMA19F['class'].value_counts()

F    39852
H     6531
G     4582
E     1942
B      432
D      202
A      165
C       67
Name: class, dtype: int64

In [9]:
sample_1rest('SMA19F', 'A', [165, 432, 67, 202, 500, 500, 500, 500]).to_csv(work_dir + 'test3_A_1rest_ready/SMA19F.tsv.gz', sep='\t', index=False, compression='gzip')

SMA19F
   class
F  39852
H   6531
G   4582
E   1942
B    432
D    202
A    165
C     67
        patient_ID
SMA19F        2866
   class
G    500
F    500
H    500
E    500
B    432
D    202
A    165
C     67
        patient_ID
SMA19F        2866
        b_class
others     2701
A           165


In [10]:
## patient SMA37F
patient_SMA37F = test3_master_new[test3_master_new['patient_ID'] == 'SMA37F']
patient_SMA37F['class'].value_counts()

F    30877
H     7497
E     3014
G     2725
B      869
A      320
C      191
D      121
Name: class, dtype: int64

In [11]:
sample_1rest('SMA37F', 'A', [320, 869, 191, 121, 500, 500, 500, 500]).to_csv(work_dir + 'test3_A_1rest_ready/SMA37F.tsv.gz', sep='\t', index=False, compression='gzip')

SMA37F
   class
F  30877
H   7497
E   3014
G   2725
B    869
A    320
C    191
D    121
        patient_ID
SMA37F        3501
   class
B    869
G    500
F    500
H    500
E    500
A    320
C    191
D    121
        patient_ID
SMA37F        3501
        b_class
others     3181
A           320


In [12]:
test3_master_new[test3_master_new['patient_ID'] == 'SMA40F']['class'].value_counts()

F    10713
H     4190
E     1905
G     1768
B     1231
A     1204
D      281
C       71
Name: class, dtype: int64

In [13]:
sample_1rest('SMA40F', 'A', [1204, 1231, 71, 281, 1905, 2000, 1768, 2000]).to_csv(work_dir + 'test3_A_1rest_ready/SMA40F.tsv.gz', sep='\t', index=False, compression='gzip')

SMA40F
   class
F  10713
H   4190
E   1905
G   1768
B   1231
A   1204
D    281
C     71
        patient_ID
SMA40F       10460
   class
H   2000
F   2000
E   1905
G   1768
B   1231
A   1204
D    281
C     71
        patient_ID
SMA40F       10460
        b_class
others     9256
A          1204


In [14]:
test3_master_new[test3_master_new['patient_ID'] == 'SMA06F']['class'].value_counts()

F    10170
H     2567
G     2497
B      693
A      551
D      496
E      453
C       32
Name: class, dtype: int64

In [15]:
sample_1rest('SMA06F', 'A', [551, 693, 32, 496, 453, 2000, 2497, 2567]).to_csv(work_dir + 'test3_A_1rest_ready/SMA06F.tsv.gz', sep='\t', index=False, compression='gzip')

SMA06F
   class
F  10170
H   2567
G   2497
B    693
A    551
D    496
E    453
C     32
        patient_ID
SMA06F        9289
   class
H   2567
G   2497
F   2000
B    693
A    551
D    496
E    453
C     32
        patient_ID
SMA06F        9289
        b_class
others     8738
A           551


In [16]:
test3_master_new[test3_master_new['patient_ID'] == 'SMA54F']['class'].value_counts()

F    8152
H    3397
G     933
E     529
B     323
A     309
D      67
C       8
Name: class, dtype: int64

In [17]:
sample_1rest('SMA54F', 'A', [309, 323, 8, 67, 529, 1000, 933, 1000]).to_csv(work_dir + 'test3_A_1rest_ready/SMA54F.tsv.gz', sep='\t', index=False, compression='gzip')

SMA54F
   class
F   8152
H   3397
G    933
E    529
B    323
A    309
D     67
C      8
        patient_ID
SMA54F        4169
   class
F   1000
H   1000
G    933
E    529
B    323
A    309
D     67
C      8
        patient_ID
SMA54F        4169
        b_class
others     3860
A           309


In [18]:
test3_master_new[test3_master_new['patient_ID'] == 'SMA60F']['class'].value_counts()

F    5663
H    2476
E    1592
G     695
A     374
C     348
B     224
D     194
Name: class, dtype: int64

In [19]:
sample_1rest('SMA60F', 'A', [374, 224, 348, 194, 1592, 2000, 695, 2476]).to_csv(work_dir + 'test3_A_1rest_ready/SMA60F.tsv.gz', sep='\t', index=False, compression='gzip')

SMA60F
   class
F   5663
H   2476
E   1592
G    695
A    374
C    348
B    224
D    194
        patient_ID
SMA60F        7903
   class
H   2476
F   2000
E   1592
G    695
A    374
C    348
B    224
D    194
        patient_ID
SMA60F        7903
        b_class
others     7529
A           374


In [20]:
test3_master_new[test3_master_new['patient_ID'] == 'SMA21F']['class'].value_counts()

F    7702
G    1124
E     361
B     127
A     102
D      30
H      26
C       4
Name: class, dtype: int64

In [21]:
sample_1rest('SMA21F', 'A', [102, 127, 4, 30, 361,2000, 1124, 26]).to_csv(work_dir + 'test3_A_1rest_ready/SMA21F.tsv.gz', sep='\t', index=False, compression='gzip')

SMA21F
   class
F   7702
G   1124
E    361
B    127
A    102
D     30
H     26
C      4
        patient_ID
SMA21F        3774
   class
F   2000
G   1124
E    361
B    127
A    102
D     30
H     26
C      4
        patient_ID
SMA21F        3774
        b_class
others     3672
A           102


 ## learning curve

In [22]:
import os
result_dir = './mIHC/results/test3_A_1rest/'

In [23]:

files = os.listdir(result_dir + 'learning_curve')
for fl in files:
    if not fl.endswith('.tabular'):
        continue
    print(fl)
    path = os.path.join(result_dir + 'learning_curve', fl)
    df = pd.read_csv(path, sep='\t')
    
    data1 = go.Scatter(
        x = df['train_sizes_abs'],
        y = df['mean_train_scores'],
        mode = "lines",
        name = "Train scores",
    )
    data2 = go.Scatter(
        x = df['train_sizes_abs'],
        y = df['mean_test_scores'],
        mode = "lines",
        name="Test scores"
    )
    layout = dict(
        xaxis=dict(
            title='No. of samples'
        ),
        yaxis=dict(
            title='Balanced Accuracy'
        )
    )
    fig = go.Figure(data=[data1, data2], layout=layout)
    iplot(fig)

SMA40F.tsv.gz.tabular


SMA19F.tsv.gz.tabular


SMA06F.tsv.gz.tabular


SMA60F.tsv.gz.tabular


SMA54F.tsv.gz.tabular


SMA37F.tsv.gz.tabular


SMA21F.tsv.gz.tabular


## hyperparameter search

In [24]:
import os
work_dir = './mIHC/results/test3_A_1rest/'
sub1 = 'RandomForestClassifier'
sub2 = 'LinearSVC'
sub3 = 'XGBClassifier'
sub4 = 'SVC'

In [25]:

def concate_best_result(folder, file_name, scorer, classifier, results):
    if not file_name.endswith('tabular'):
        return results
    path = os.path.join(folder, file_name)
    res = pd.read_csv(path, sep='\t')
    
    res_sort = res.sort_values(['mean_test_'+scorer, 'std_test_'+scorer], ascending=[False, True])
    res_best = res_sort[['mean_test_'+scorer, 'std_test_'+scorer,'params']].head(1).reset_index(drop=True)
    res_best.insert(loc=0, column='dataset', value=file_name[:-11])
    res_best.insert(loc=0, column='classifier', value=classifier)
    if res_best['mean_test_'+scorer].isnull().any():
        print("%s got null %s" %(file_name, scorer))
    else:
        if results is None:
            results = res_best
        else:
            results = results.append(res_best, ignore_index=True)
    return results

In [26]:
# best f1_macro scores
files1 = os.listdir(work_dir+sub1)
files2 = os.listdir(work_dir+sub2)
files3 = os.listdir(work_dir+sub3)
files4 = os.listdir(work_dir+sub4)
results_f1 = None
scorer = 'f1_macro'
for fl in files1:
    results_f1 = concate_best_result(work_dir+sub1, fl, scorer, 'RandomForestClassifier', results_f1)

for fl in files2:
    results_f1 = concate_best_result(work_dir+sub2, fl, scorer, 'LinearSVC', results_f1)
    
for fl in files3:
    results_f1 = concate_best_result(work_dir+sub3, fl, scorer, 'XGBClassifier', results_f1)

for fl in files4:
    results_f1 = concate_best_result(work_dir+sub4, fl, scorer, 'SVC', results_f1)


results_f1 = results_f1.sort_values(['classifier', 'dataset'])
results_f1

,classifier,dataset,mean_test_f1_macro,std_test_f1_macro,params
9,LinearSVC,SMA06F.tsv,0.933926,0.019060,"{u'estimator__dual': False, u'estimator__C': 1..."
8,LinearSVC,SMA19F.tsv,0.880744,0.021183,"{u'estimator__dual': False, u'estimator__C': 1..."
13,LinearSVC,SMA21F.tsv,0.865713,0.028544,"{u'estimator__dual': False, u'estimator__C': 1..."
12,LinearSVC,SMA37F.tsv,0.897069,0.013290,"{u'estimator__dual': False, u'estimator__C': 1..."
7,LinearSVC,SMA40F.tsv,0.957775,0.001134,"{u'estimator__dual': False, u'estimator__C': 1..."
11,LinearSVC,SMA54F.tsv,0.953715,0.009845,"{u'estimator__dual': False, u'estimator__C': 1..."
10,LinearSVC,SMA60F.tsv,0.854935,0.017898,"{u'estimator__dual': False, u'estimator__C': 1..."
2,RandomForestClassifier,SMA06F.tsv,0.985695,0.006974,"{u'estimator__class_weight': 'balanced', u'est..."
1,RandomForestClassifier,SMA19F.tsv,0.983635,0.010505,{u'estimator__class_weight': 'balanced_subsamp...
6,RandomForestClassifier,SMA21F.tsv,0.962757,0.022070,{u'estimator__class_weight': 'balanced_subsamp...


In [27]:
# best b_accuracy scores
files1 = os.listdir(work_dir+sub1)
files2 = os.listdir(work_dir+sub2)
files3 = os.listdir(work_dir+sub3)
files4 = os.listdir(work_dir+sub4)
results_b_accuracy = None
scorer = 'balanced_accuracy'
for fl in files1:
    results_b_accuracy = concate_best_result(work_dir+sub1, fl, scorer, 'RandomForestClassifier', results_b_accuracy)

for fl in files2:
    results_b_accuracy = concate_best_result(work_dir+sub2, fl, scorer, 'LinearSVC', results_b_accuracy)
    
for fl in files3:
    results_b_accuracy = concate_best_result(work_dir+sub3, fl, scorer, 'XGBClassifier', results_b_accuracy)

for fl in files4:
    results_b_accuracy = concate_best_result(work_dir+sub4, fl, scorer, 'SVC', results_b_accuracy)


results_b_accuracy = results_b_accuracy.sort_values(['classifier', 'dataset'])
results_b_accuracy

,classifier,dataset,mean_test_balanced_accuracy,std_test_balanced_accuracy,params
9,LinearSVC,SMA06F.tsv,0.926779,0.017274,"{u'estimator__dual': False, u'estimator__C': 1..."
8,LinearSVC,SMA19F.tsv,0.887332,0.054253,"{u'estimator__dual': False, u'estimator__C': 1..."
13,LinearSVC,SMA21F.tsv,0.853085,0.051217,"{u'estimator__dual': False, u'estimator__C': 1..."
12,LinearSVC,SMA37F.tsv,0.882147,0.017589,"{u'estimator__dual': False, u'estimator__C': 1..."
7,LinearSVC,SMA40F.tsv,0.948846,0.006015,"{u'estimator__dual': False, u'estimator__C': 1..."
11,LinearSVC,SMA54F.tsv,0.951054,0.011723,"{u'estimator__dual': False, u'estimator__C': 1..."
10,LinearSVC,SMA60F.tsv,0.835238,0.033167,"{u'estimator__dual': False, u'estimator__C': 1..."
2,RandomForestClassifier,SMA06F.tsv,0.981155,0.008768,{u'estimator__class_weight': 'balanced_subsamp...
1,RandomForestClassifier,SMA19F.tsv,0.978224,0.015606,{u'estimator__class_weight': 'balanced_subsamp...
6,RandomForestClassifier,SMA21F.tsv,0.945551,0.036513,{u'estimator__class_weight': 'balanced_subsamp...


In [28]:
data1 = go.Bar(
    x = results_f1[results_f1['classifier'] == 'LinearSVC']['dataset'],
    y = results_f1[results_f1['classifier'] == 'LinearSVC']['mean_test_f1_macro'],
    name = 'LiSVC_f1_macro'
)
data2 = go.Bar(
    x = results_f1[results_f1['classifier'] == 'RandomForestClassifier']['dataset'],
    y = results_f1[results_f1['classifier'] == 'RandomForestClassifier']['mean_test_f1_macro'],
    name = 'RF_f1_macro'
)
data3 = go.Bar(
    x = results_f1[results_f1['classifier'] == 'XGBClassifier']['dataset'],
    y = results_f1[results_f1['classifier'] == 'XGBClassifier']['mean_test_f1_macro'],
    name = 'XGBC_f1_macro'
)
data4 = go.Bar(
    x = results_f1[results_f1['classifier'] == 'SVC']['dataset'],
    y = results_f1[results_f1['classifier'] == 'SVC']['mean_test_f1_macro'],
    name = 'SVC_f1_macro'
)


data5 = go.Bar(
    x = results_b_accuracy[results_b_accuracy['classifier'] == 'LinearSVC']['dataset'],
    y = results_b_accuracy[results_b_accuracy['classifier'] == 'LinearSVC']['mean_test_balanced_accuracy'],
    name = 'LiSVC_b_accu'
)


data6 = go.Bar(
    x = results_b_accuracy[results_b_accuracy['classifier'] == 'RandomForestClassifier']['dataset'],
    y = results_b_accuracy[results_b_accuracy['classifier'] == 'RandomForestClassifier']['mean_test_balanced_accuracy'],
    name = 'RF_b_accu'
)

data7 = go.Bar(
    x = results_b_accuracy[results_b_accuracy['classifier'] == 'XGBClassifier']['dataset'],
    y = results_b_accuracy[results_b_accuracy['classifier'] == 'XGBClassifier']['mean_test_balanced_accuracy'],
    name = 'XGBC_b_accu'
)
data8 = go.Bar(
    x = results_b_accuracy[results_b_accuracy['classifier'] == 'SVC']['dataset'],
    y = results_b_accuracy[results_b_accuracy['classifier'] == 'SVC']['mean_test_balanced_accuracy'],
    name = 'SVC_b_accu'
)


layout = go.Layout(
    xaxis=dict(
        title='Dataset'
    ),
    yaxis=dict(
        title='Performance score'
    ),
    barmode = 'group'
)
fig = go.Figure(data=[data1, data2, data3, data4], layout=layout)
iplot(fig)

fig = go.Figure(data=[data5, data6, data7, data8], layout=layout)
iplot(fig)
# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [29]:
trace1 = {
    "type": 'violin',
    "x": results_f1['classifier'],
    "y": results_f1['mean_test_f1_macro'],
    "legendgroup": 'f1_macro',
    "scalegroup": 'f1_macro',
    "name": 'f1_macro',
    "box": {
        "visible": True
    },
    "meanline": {
        "visible": True
    },
    "line": {
        "color": 'blue'
    }
}

trace2 = {
    "type": 'violin',
    "x": results_b_accuracy['classifier'],
    "y": results_b_accuracy['mean_test_balanced_accuracy'],
    "legendgroup": 'b_accuracy',
    "scalegroup": 'b_accuracy',
    "name": 'b_accuracy',
    "box": {
        "visible": True
    },
    "meanline": {
        "visible": True
    },
    "line": {
        "color": 'pink'
    }
}

layout = {
    "yaxis": {
        "zeroline": False,
    },
    "violinmode": 'group'
}
fig = go.Figure(data=[trace1, trace2], layout=layout)
iplot(fig)
# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/